In [94]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
import json
import requests
import pandas as pd

In [85]:
mongo = MongoClient(port=27017)
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'establishments', 'fruits_db', 'fruitsdb', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food', 'weather_database']


In [89]:
mongo = MongoClient('mongodb://localhost:27017')
db = mongo['player2_weather']
sanibel_air = db['sanibel_air']
endpoint = 'https://api.stormglass.io/v2/weather/point'
params = {
            'lat': 26.443398,
            'lng': -82.111511,
            'params': 'airTemperature',
            'start': '2023-09-23',
            'end': '2023-09-30'
            
        }
headers = {
        'Authorization': 'f83644f0-b99c-11ee-b7a8-0242ac130002-f8364590-b99c-11ee-b7a8-0242ac130002'
    }
response = requests.get(endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()
sanibel_air.insert_one(data)
if sanibel_air.count_documents({}) > 0:
    print("Data inserted into sanibel_data collection.")
else:
    print("Failed to insert data.")


Data inserted into sanibel_data collection.


In [90]:
data = sanibel_air.find()
for document in data:
    pprint(document)

{'_id': ObjectId('65b9539ce5af0f131d426418'),
 'hours': [{'airTemperature': {'noaa': 28.42, 'sg': 28.42},
            'time': '2021-09-23T00:00:00+00:00'},
           {'airTemperature': {'noaa': 28.25, 'sg': 28.25},
            'time': '2021-09-23T01:00:00+00:00'},
           {'airTemperature': {'noaa': 28.08, 'sg': 28.08},
            'time': '2021-09-23T02:00:00+00:00'},
           {'airTemperature': {'noaa': 27.91, 'sg': 27.91},
            'time': '2021-09-23T03:00:00+00:00'},
           {'airTemperature': {'noaa': 27.56, 'sg': 27.56},
            'time': '2021-09-23T04:00:00+00:00'},
           {'airTemperature': {'noaa': 27.21, 'sg': 27.21},
            'time': '2021-09-23T05:00:00+00:00'},
           {'airTemperature': {'noaa': 26.86, 'sg': 26.86},
            'time': '2021-09-23T06:00:00+00:00'},
           {'airTemperature': {'noaa': 26.79, 'sg': 26.79},
            'time': '2021-09-23T07:00:00+00:00'},
           {'airTemperature': {'noaa': 26.72, 'sg': 26.72},
            't

In [99]:
data = list(sanibel_air.find())
pprint(data)
json_data = json.dumps(data, default=str)
with open('Resources/sanibel_air.json', 'w') as json_file:
    json_file.write(json_data)

[{'_id': ObjectId('65b9539ce5af0f131d426418'),
  'hours': [{'airTemperature': {'noaa': 28.42, 'sg': 28.42},
             'time': '2021-09-23T00:00:00+00:00'},
            {'airTemperature': {'noaa': 28.25, 'sg': 28.25},
             'time': '2021-09-23T01:00:00+00:00'},
            {'airTemperature': {'noaa': 28.08, 'sg': 28.08},
             'time': '2021-09-23T02:00:00+00:00'},
            {'airTemperature': {'noaa': 27.91, 'sg': 27.91},
             'time': '2021-09-23T03:00:00+00:00'},
            {'airTemperature': {'noaa': 27.56, 'sg': 27.56},
             'time': '2021-09-23T04:00:00+00:00'},
            {'airTemperature': {'noaa': 27.21, 'sg': 27.21},
             'time': '2021-09-23T05:00:00+00:00'},
            {'airTemperature': {'noaa': 26.86, 'sg': 26.86},
             'time': '2021-09-23T06:00:00+00:00'},
            {'airTemperature': {'noaa': 26.79, 'sg': 26.79},
             'time': '2021-09-23T07:00:00+00:00'},
            {'airTemperature': {'noaa': 26.72, 'sg': 26.7

In [101]:
data = pd.read_json('Resources/sanibel_air.json')
data.to_csv('Resources/sanibel_air.csv', index = False)